In [1]:
import networkx as nx
import csv
import numpy as np
from random import randint
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB, MultinomialNB,BernoulliNB
from sklearn import tree
import xgboost as xg
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer
import matplotlib.pyplot as plt
import nltk
from nltk.probability import FreqDist
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [3]:
G = nx.read_edgelist('edgelist.txt', delimiter=',', create_using=nx.Graph(), nodetype=int)
nodes = list(G.nodes())
n = G.number_of_nodes()
m = G.number_of_edges()
print('Number of nodes:', n)
print('Number of edges:', m)
print(nx.is_connected(G))


Number of nodes: 138499
Number of edges: 1091955
False


In [4]:
subgraph=G
for i in range (10000,n):
    subgraph.remove_node(i)
    
print(nx.info(subgraph))

#nx.draw(subgraph)

#FIND GIANT COMPONENT
#components = nx.connected_components(G)
#largest_component = max(components, key=len)

#subgraph = G.subgraph(largest_component)
#diameter = nx.diameter(subgraph)
#print("Network diameter of largest component:", diameter)

#triadic_closure = nx.transitivity(G)
#print("Triadic closure:", triadic_closure)

Graph with 10000 nodes and 56226 edges


In [9]:
#Find density on G and on the subgraph(they prove to be the same)
G_density=nx.density(G)
print("G density: ",G_density)

sub_density=nx.density(subgraph)
print("sub density: ",sub_density)

#Finds all possible paths and chooses the shortest one
short_path=nx.shortest_path(G,source=154,target=6551)
print("Shortest path between these two is:", short_path)

triadic_closure = nx.transitivity(G)
print("Triadic closure:", triadic_closure)

bridges=list(nx.bridges(subgraph))

for bridge in bridges:
    print(bridge)
    
print("------------")
#local_bridges=list(nx.local_bridges(subgraph))

#for local in local_bridges:
    #print(local)

cliques = list(nx.find_cliques(G))

clique_number = len(list(cliques))
print(clique_number)




print("%.4f"% nx.degree_assortativity_coefficient(G))

G density:  0.0011246324632463247
sub density:  0.0011246324632463247
Shortest path between these two is: [154, 66, 245, 4860, 6551]
Triadic closure: 0.02696968287398654
(1, 3)
(1, 7)
(1, 15)
(1, 17)
(1, 19)
(2, 27)
(2, 29)
(2, 46)
(2, 56)
(7, 8)
(43, 412)
(43, 428)
(4, 5202)
(798, 4014)
(798, 4137)
(798, 4141)
(798, 4179)
(798, 4301)
(798, 4431)
(798, 4541)
(798, 4554)
(798, 4584)
(407, 47)
(407, 479)
(407, 480)
(407, 484)
(407, 488)
(407, 490)
(407, 492)
(407, 497)
(407, 499)
(407, 501)
(407, 508)
(407, 510)
(407, 512)
(407, 513)
(407, 517)
(407, 518)
(407, 521)
(407, 526)
(407, 527)
(407, 534)
(407, 535)
(407, 536)
(407, 537)
(407, 540)
(407, 546)
(407, 548)
(407, 549)
(407, 553)
(407, 554)
(407, 561)
(407, 562)
(407, 566)
(407, 567)
(407, 569)
(407, 578)
(407, 581)
(407, 582)
(407, 589)
(407, 596)
(407, 599)
(407, 605)
(407, 607)
(407, 609)
(407, 613)
(407, 614)
(407, 617)
(407, 621)
(407, 623)
(407, 625)
(407, 634)
(407, 636)
(407, 637)
(407, 638)
(407, 643)
(407, 650)
(407, 652)


37302
-0.0812


In [ ]:
#Find degree 
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree')
print(G.nodes[2112])

from operator import itemgetter
sorted_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)
print("Top 20 nodes by degree:")
for d in sorted_degree[:20]:
    print(d)
    
    
betweenness_dict = nx.betweenness_centrality(subgraph) # Run betweenness centrality
eigenvector_dict = nx.eigenvector_centrality(subgraph) # Run eigenvector centrality

# Assign each to an attribute in your network
nx.set_node_attributes(subgraph, betweenness_dict, 'betweenness')
nx.set_node_attributes(subgraph, eigenvector_dict, 'eigenvector')

sorted_betweenness = sorted(betweenness_dict.items(), key=itemgetter(1), reverse=True)

print("Top 20 nodes by betweenness centrality:")
for b in sorted_betweenness[:20]:
    print(b)

In [ ]:
#Find the communities in the graph
from networkx.algorithms import community
communities = community.greedy_modularity_communities(subgraph)
print( len(communities))

    
for i,c in enumerate(communities): # Loop through the list of communities
   # if len(c) > 2: # Filter out modularity classes with 2 or fewer nodes
        print('Class '+str(i)+':',len(c), list(c)) # Print out the classes and their members

In [6]:
# Read the abstract of each paper
abstracts = dict()
with open('abstracts.txt', 'r',encoding='utf8') as f:
    for line in f:
        node, abstract = line.split('|--|')
        abstracts[int(node)] = abstract

In [7]:
import bs4 as bs
import re
import gensim
#nltk.download('stopwords')

from nltk.corpus import stopwords


from nltk.tokenize import sent_tokenize, word_tokenize
import warnings

warnings.filterwarnings(action = 'ignore')

import gensim
from gensim.models import Word2Vec


string=str(abstracts[0])
data=[]
# iterate through each sentence in the file
for i in sent_tokenize(string):
    temp = []
     
    # tokenize the sentence into words
    for j in word_tokenize(i):
        temp.append(j.lower())
 
    data.append(temp)
    
# Create CBOW model
model1 = gensim.models.Word2Vec(data, min_count = 1,vector_size = 100, window = 5)
print(data)
print("Cosine similarity between 'development' " +"and 'placed' - CBOW : ", model1.wv.similarity("development", "placed"))
print(model1.wv.doesnt_match(["development", "recommendations","placed" ]))#Word that doesn't match
print(model1.wv.most_similar(positive=["development", "recommendations", "placed"], negative=["inside"], topn=3))

[['the', 'development', 'of', 'an', 'automated', 'system', 'for', 'the', 'quality', 'assessment', 'of', 'aerodrome', 'ground', 'lighting', '(', 'agl', ')', ',', 'in', 'accordance', 'with', 'associated', 'standards', 'and', 'recommendations', ',', 'is', 'presented', '.'], ['the', 'system', 'is', 'composed', 'of', 'an', 'image', 'sensor', ',', 'placed', 'inside', 'the', 'cockpit', 'of', 'an', 'aircraft', 'to', 'record', 'images', 'of', 'the', 'agl', 'during', 'a', 'normal', 'descent', 'to', 'an', 'aerodrome', '.'], ['a', 'model-based', 'methodology', 'is', 'used', 'to', 'ascertain', 'the', 'optimum', 'match', 'between', 'a', 'template', 'of', 'the', 'agl', 'and', 'the', 'actual', 'image', 'data', 'in', 'order', 'to', 'calculate', 'the', 'position', 'and', 'orientation', 'of', 'the', 'camera', 'at', 'the', 'instant', 'the', 'image', 'was', 'acquired', '.'], ['the', 'camera', 'position', 'and', 'orientation', 'data', 'are', 'used', 'along', 'with', 'the', 'pixel', 'grey', 'level', 'for', '

In [8]:
for node in abstracts:
    abstracts[node] = tuple(abstracts[node].split())
    
#from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler()
#data_rescaled = scaler.fit_transform(str(abstracts))

In [ ]:
#Implement SentimentIntensityAnalyzer to find if the text is positive,negative or neutral
from nltk.sentiment import SentimentIntensityAnalyzer
#nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

total=tuple()
for i in range (0,n-1):
    total+=abstracts[i]

string=str(abstracts[0])    #For a quicker check
#string=str(total)
sia.polarity_scores(string)

In [10]:
from nltk.probability import FreqDist
#print (fdist)
fdist = FreqDist(abstracts)

fdist1 = fdist.most_common(5)
fdist1

[(68654,
  ('�',
   'Although',
   'one',
   'of',
   'the',
   'fundamental',
   'goals',
   'of',
   'AI',
   'is',
   'to',
   'understand',
   'and',
   'develop',
   'intelligent',
   'systems',
   'that',
   'have',
   'all',
   'the',
   'capabilities',
   'of',
   'humans,',
   'there',
   'is',
   'little',
   'active',
   'research',
   'directly',
   'pursuing',
   'this',
   'goal.',
   'We',
   'propose',
   'that',
   'AI',
   'for',
   'interactive',
   'computer',
   'games',
   'is',
   'an',
   'emerging',
   'application',
   'area',
   'in',
   'which',
   'this',
   'goal',
   'of',
   'human-level',
   'AI',
   'can',
   'successfully',
   'be',
   'pursued.',
   'Interactive',
   'computer',
   'games',
   'have',
   'increasingly',
   'complex',
   'and',
   'realistic',
   'worlds',
   'and',
   'increasingly',
   'complex',
   'and',
   'intelligent',
   'computer-controlled',
   'characters.',
   'In',
   'this',
   'article,',
   'we',
   'further',
   'moti

In [11]:
authors = dict()
from collections import defaultdict
with open('authors.txt', 'r',encoding="utf8") as f:
    for line in f:
        node, author = line.split('|--|')
        authors[int(node)] = author
        
        

#Find the authors that are mostly in the list
fdist = FreqDist(authors)

fdist1 = fdist.most_common(5)
fdist1

[(20300, 'Žiga Emeršič,Vitomir Štruc,Peter Peer\n'),
 (97681,
  'Želmira Tóthová,Jaroslav Polec,Tatiana Orgoniková,Lenka Krulikovská\n'),
 (110528, 'Željko Agić,Barbara Plank,Anders Søgaard\n'),
 (114532, 'Żeljko Agić,Nikola Ljubešić,Danijela Merkler\n'),
 (51933, 'Żeljko Agić,Dirk Hovy,Anders Sogaard\n')]

In [12]:
# its class label is 1 if it corresponds to an edge and 0, otherwise.
# Use the following 3 features for each pair of nodes:
# (1) sum of number of unique terms of the two nodes' abstracts
# (2) absolute value of difference of number of unique terms of the two nodes' abstracts
# (3) number of common terms between the abstracts of the two nodes
X_train = np.zeros((2*m, 4))
y_train = np.zeros(2*m)
n = G.number_of_nodes()
max=-1

for i,edge in enumerate(G.edges()):
    # an edge
    X_train.at[i,0] = len(abstracts[edge[0]]) + len(abstracts[edge[1]])
    X_train[i,1] = abs(len(abstracts[edge[0]]) - len(abstracts[edge[1]]))
    #X_train[i,2] = len((abstracts[edge[0]]).intersection(abstracts[edge[1]]))
    X_train[i,2]=len(list(set(abstracts[edge[0]])&set(abstracts[edge[1]])))
    X_train[i,3]=len(abstracts[edge[0]])/(len(abstracts[edge[1]])+1)
    #X_train[i,3]=abstracts[edge[0]]+abstracts[edge[1]]
    y_train[i] = 1
    
    

    # a randomly generated pair of nodes
    n1 = randint(0, n-1)
    n2 = randint(0, n-1)
    X_train[m+i,0] = len(abstracts[n1]) + len(abstracts[n2])
    X_train[m+i,1] = abs(len(abstracts[n1]) - len(abstracts[n2]))
    #X_train[m+i,2] = len(abstracts[n1].intersection(abstracts[n2]))
    X_train[m+i,2]=len(list(set(abstracts[n1])&set(abstracts[n2])))
    X_train[m+i,3]=len(abstracts[n1])/(len(abstracts[n2])+1)
    #X_train[m+i,3]=abstracts[edge[0]]+abstracts[edge[1]]
    y_train[m+i] = 0
    
    
"""
#Scaling by finding max and dividing each element by the max
for i in range(0,m-1):
    for j in range(0,3):
        if (X_train[i,j]>max):
            max=X_train[i,j]
        
print(max)

for i in range(0,m-1):
    for j in range(0,3):
        X_train[i,j]=X_train[i,j]/max
        
"""

'\n#Scaling by finding max and dividing each element by the max\nfor i in range(0,m-1):\n    for j in range(0,3):\n        if (X_train[i,j]>max):\n            max=X_train[i,j]\n        \nprint(max)\n\nfor i in range(0,m-1):\n    for j in range(0,3):\n        X_train[i,j]=X_train[i,j]/max\n        \n'

In [6]:
#Feature selection on X_train
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X_train, y_train)
model = SelectFromModel(lsvc, prefit=True)
X_train = model.transform(X_train)


(150, 3)

In [13]:
print('Size of training matrix:', X_train.shape)

Size of training matrix: (2183910, 4)


In [14]:
# Read test data. Each sample is a pair of nodes
node_pairs = list()
with open('test.txt', 'r') as f:
    for line in f:
        t = line.split(',')
        node_pairs.append((int(t[0]), int(t[1])))
node_pairs=tuple(node_pairs)




#print(node_pairs[0])

In [15]:
# Create the test matrix. Use the same 4 features as above
X_test = np.zeros((len(node_pairs), 4))
for i,node_pair in enumerate(node_pairs):
    X_test[i,0] = len(abstracts[node_pair[0]]) + len(abstracts[node_pair[1]])
    X_test[i,1] = abs(len(abstracts[node_pair[0]]) - len(abstracts[node_pair[1]]))
   # X_test[i,2] = len(abstracts[node_pair[0]].intersection(abstracts[node_pair[1]]))
    X_test[i,2]=len(list(set(abstracts[node_pair[0]])&set(abstracts[node_pair[1]])))
    X_test[i,3]=len(abstracts[node_pair[0]])/(len(abstracts[node_pair[1]])+1)
    #X_test[i,3]=abstracts[node_pair[0]]+abstracts[node_pair[1]]
    
"""
max=-1
for i in range (0,len(node_pairs)):
    for j in range (0,3):
        if (X_test[i,j]>max):
            max=X_test[i,j]
            
for i in range(0,len(node_pairs)):
    for j in range(0,3):
        X_test[i,j]=X_test[i,j]/max
            
print('Size of test matrix:', X_test.shape)
#X_test=pca.fit_transform(X_test)
"""

#scaler=preprocessing.StandardScaler().fit(X_test)
#X_test=scaler.transform(X_test)

"\nmax=-1\nfor i in range (0,len(node_pairs)):\n    for j in range (0,3):\n        if (X_test[i,j]>max):\n            max=X_test[i,j]\n            \nfor i in range(0,len(node_pairs)):\n    for j in range(0,3):\n        X_test[i,j]=X_test[i,j]/max\n            \nprint('Size of test matrix:', X_test.shape)\n#X_test=pca.fit_transform(X_test)\n"

In [16]:
from sklearn.preprocessing import MinMaxScaler
"""

#Scale data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
y_train=scaler.fit_transform(y_train.reshape(-1,1))
X_test=scaler.fit_transform(X_test)
"""

"""
#Apply PCA
pca=PCA(n_components=0.99)
X_train=pca.fit_transform(X_train,y_train)
y_train=pca.fit_transform(y_train.reshape(-1, 1))
X_test=pca.fit_transform(X_test)
"""


"""
#Apply LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components = 2)
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)
"""
X_train, y_train = shuffle(X_train, y_train)

print(X_train.shape)
print(y_train.shape)


(2183910, 4)
(2183910,)


In [13]:
from sklearn import preprocessing
from sklearn import utils

lab = preprocessing.LabelEncoder()
y_train = lab.fit_transform(y_train.ravel())


In [ ]:
import time

# Use logistic regression to predict if two nodes are linked by an edge
clf = LogisticRegression(class_weight='balanced', max_iter=20,random_state=1234,solver='liblinear')
#clf = MLPClassifier(hidden_layer_sizes=(6,5),
                   # random_state=5,
                    #verbose=True,
                    #learning_rate_init=0.01)
## ΝΑ ΔΟΚΙΜΑΣΩ ΚΑΙ RNNs,CNNs
#clf=LogisticRegression()
#clf=tree.DecisionTreeClassifier()
start_time = time.time()
clf.fit(X_train, y_train)
end_time = time.time()
print("Logistic Time: ", end_time-start_time, "score: ", clf.score(X_train,y_train))

y_pred = clf.predict_proba(X_test)
y_pred = y_pred[:,1]
print(y_pred)

print("Logistic R. score: " , clf.score(X_train,y_train))
print(classification_report(y_train,clf.predict(X_train)))
print(confusion_matrix(y_train,clf.predict(X_train)))






Bernoulli=BernoulliNB()
Multinomial=MultinomialNB()
Gaussian=GaussianNB()
knn=KNeighborsClassifier(n_neighbors = 1)
forest=RandomForestClassifier(n_estimators = 100) 
MLP= MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
Adaboost = AdaBoostClassifier(n_estimators=100, random_state=0)
SVClf = SVC(kernel='linear')
LinearD=LDA()

start_time = time.time()
Bernoulli.fit(X_train, y_train)
end_time = time.time()
print("Bernoulli Time: ", end_time-start_time, " score: " , Bernoulli.score(X_train,y_train))

start_time = time.time()
Multinomial.fit(X_train, y_train)
end_time = time.time()
print("Multinomial Time: ", end_time-start_time," score: " , Multinomial.score(X_train,y_train))

start_time = time.time()
Gaussian.fit(X_train, y_train)
end_time = time.time()
print("Gaussian Time: ", end_time-start_time," score: " , Gaussian.score(X_train,y_train))

start_time = time.time()
knn.fit(X_train, y_train)
end_time = time.time()
print("KNN Time: ", end_time-start_time," score: " , knn.score(X_train,y_train))

start_time = time.time()
forest.fit(X_train, y_train)
end_time = time.time()
print("Random forest Time: ", end_time-start_time," score: " , forest.score(X_train,y_train))

start_time = time.time()
MLP.fit(X_train, y_train)
end_time = time.time()
print("MLP Time: ", end_time-start_time, " score: " , MLP.score(X_train,y_train))

start_time = time.time()
Adaboost.fit(X_train, y_train)
end_time = time.time()
print("Adaboost Time: ", end_time-start_time," score: " , Adaboost.score(X_train,y_train))

start_time = time.time()
SVClf.fit(X_train, y_train)
end_time = time.time()
print("SVM Time: ", end_time-start_time," score: " , SVClf.score(X_train,y_train))

start_time = time.time()
LinearD.fit(X_train, y_train)
end_time = time.time()
print("LDA Time: ", end_time-start_time, " score: " , LinearD.score(X_train,y_train))





In [ ]:
#GRIDSEARCHCV for MLP

from sklearn.metrics import make_scorer,fbeta_score
GRID = [
    {'scaler': [StandardScaler()],
     'estimator': [MLPClassifier(random_state=101)],
     'estimator__solver': ['adam'],
     'estimator__learning_rate_init': [0.0001],
     'estimator__max_iter': [300],
     #'estimator__hidden_layer_sizes': [(500, 400, 300, 200, 100), (400, 400, 400, 400, 400), (300, 300, 300, 300, 300), (200, 200, 200, 200, 200)],
     'estimator__activation': ['logistic', 'tanh', 'relu'],
     'estimator__alpha': [ 0.001, 0.005]
    # 'estimator__early_stopping': [True, False]
     }
]

PIPELINE = Pipeline([('scaler', None), ('estimator', MLPClassifier())])



grid_search = GridSearchCV(estimator=PIPELINE, param_grid=GRID, 
                            scoring=make_scorer(fbeta_score, beta=2),# average='macro'), 
                            n_jobs=-1, refit=True, verbose=1, 
                            return_train_score=False)

grid_search.fit(X_train, y_train)
print(grid_search.score(X_train,y_train))

In [20]:
#GridSearchCV for logistic regression
parameter_grid_logistic_regression = {
    'max_iter': [ 20,50],                      # Number of iterations
    'solver': [ 'liblinear', 'saga'],   # Algorithm to use for optimization
    'class_weight': ['balanced']                                    # Troubleshoot unbalanced data sampling
} 


logistic_Model_grid = GridSearchCV(estimator=LogisticRegression(random_state=1234), param_grid=parameter_grid_logistic_regression, verbose=1, 
                    cv=10, n_jobs=-1)

logistic_Model_grid.fit(X_train, y_train)

print("Best score for the model after tuning is: ",logistic_Model_grid.best_score_)
print("Best parameters for the model is :",logistic_Model_grid.best_estimator_)

Fitting 10 folds for each of 4 candidates, totalling 40 fits
Best score for the model after tuning is:  0.7177850735607236
Best parameters for the model is : LogisticRegression(class_weight='balanced', max_iter=50, random_state=1234,
                   solver='liblinear')


In [ ]:
y_pred = forest.predict_proba(X_test)
y_pred = y_pred[:,1]

In [12]:
# Write predictions to a file
predictions = zip(range(len(y_pred)), y_pred)
with open("submission_text.csv","w") as pred:
    csv_out = csv.writer(pred)
    csv_out.writerow(['id','predicted'])
    for row in predictions:
        csv_out.writerow(row)